#### 梯度下降法

是一个通用的优化算法来帮助机器学习算法求解出最优解的。

所有优化算法的目的都是以最快的速度把模型参数θ求解出来

之前利用解析解公式一步求出参数的最优解，它的复杂度可以达到N的3次方，并不是常用的手段，而梯度下降法是一点点逼近最优解

<img src="./Img/6.png" width="40%" height="40%">

这个曲线其实就是损失函数，直到我们得到的MSE最小的时候，计算出来的θ就是我们的最优解